In [81]:
import os

In [82]:
os.getcwd()

'C:\\Users\\ganreddy'

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

C:\Users\ganreddy\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
train_data_df = pd.read_csv('C:\\Users\\ganreddy\\Documents\\R Scripts\\Benz\\train.csv')

In [15]:
test_data_df = pd.read_csv('C:\\Users\\ganreddy\\Documents\\R Scripts\\Benz\\test.csv')

In [16]:
Data = train_data_df.append(test_data_df, ignore_index=True)

In [20]:
Data['X0']='X0'+Data['X0']
Data['X1']='X1'+Data['X1']
Data['X2']='X2'+Data['X2']
Data['X3']='X3'+Data['X3']
Data['X4']='X4'+Data['X4']
Data['X5']='X5'+Data['X5']
Data['X6']='X6'+Data['X6']
Data['X8']='X8'+Data['X8']

In [22]:
# X0,X1,X2,X3,X4,X5,X6,X7,X8
Data_X0 = pd.get_dummies(Data['X0'])
Data_X1 = pd.get_dummies(Data['X1'])
Data_X2 = pd.get_dummies(Data['X2'])
Data_X3 = pd.get_dummies(Data['X3'])
Data_X4 = pd.get_dummies(Data['X4'])
Data_X5 = pd.get_dummies(Data['X5'])
Data_X6 = pd.get_dummies(Data['X6'])
Data_X8 = pd.get_dummies(Data['X8'])

In [24]:

#Train
Data = Data.join(Data_X0)
Data = Data.join(Data_X1)
Data = Data.join(Data_X2)
Data = Data.join(Data_X3)
Data = Data.join(Data_X4)
Data = Data.join(Data_X5)
Data = Data.join(Data_X6)
Data = Data.join(Data_X8)


#Drop the Remaining
#train_data_df.drop(train_data_df.columns[[2,3,4,5,6,7,8,9]], axis=1, inplace=True)



In [30]:
Data.drop(['X0', 'X1','X2','X3','X4','X5','X6','X8'], axis=1, inplace=True)

In [48]:
train_data_df=Data[Data['y'].notnull()]
test_data_df=Data[Data['y'].isnull()]

In [49]:
output = pd.Series(train_data_df.y, dtype="float")
train_data_df = train_data_df.drop('y',1)
train_data_df = train_data_df.drop('ID',1)

In [50]:
id = test_data_df.ID
test_data_df = test_data_df.drop('y',1)
test_data_df = test_data_df.drop('ID',1)

In [51]:
test_data_df = test_data_df.astype('float')
train_data_df = train_data_df.astype('float')

In [52]:
xgb_tr = xgb.DMatrix(train_data_df, label=output, missing=np.NaN)
xgb_ts = xgb.DMatrix(test_data_df, missing=np.NaN)

In [53]:
############## Prediction model #################
param = {}
#param['objective'] = 'binary:logitraw'
param['objective'] = 'reg:linear'
param['eta'] = 0.1
param['max_delta_step'] = 1000
param['max_depth'] = 5    #20
# param['num_class'] = 2
param['lambda']=0.001
param['subsample'] = 0.85
# param['colsample_bytree'] = 1
param['gamma'] = 1
param['min_child_weight'] = 17
num_round = 2000   #2000


In [54]:
gbm = xgb.train(param,xgb_tr,num_round)

In [55]:
test_pred = gbm.predict(xgb_ts)

In [59]:
test_pred

array([  78.86367035,   93.24419403,   79.40079498, ...,   88.70170593,
        110.10678864,   91.01621246], dtype=float32)

In [75]:
op = []
for i  in zip(id,test_pred):
    op.append(i)

In [78]:
op_df = pd.DataFrame(op,columns=['ID','y'])

In [80]:
op_df.to_csv("C:\\Users\\ganreddy\\Documents\\R Scripts\\Benz\\submission.csv")